## Forward autodiff in pyhton
Can be divided into two parts:
1) implementing the **Dual number** algebra (a+εb) where 'ε' is a nipotent number (ε != 0 but ε^2=0)
2) implementing computational graphs using the dual number algebra.

Later I realised that there was no need to implement a graph to implement fwd autodiff, they are completely independent concepts. Infact inorder to compute partial derivatives, fwd autodiff only requires computing the value of a function using dual number algebra at the required point.

In [2]:
import numpy as np

In [3]:
# the Dual number class
class dual:
    def __init__(self, a, b=0):
        self.first=a
        self.second=b
    
    #None of the basic algebraic operations defined below for dual number change the original dual numbers passed to the overloaded functions
    #__add__ is a magic method which helps in operator overloading in pyhton
    def __add__(self, num):
        if isinstance(num, int):
            num=dual(num)
        return dual(self.first+num.first, self.second+num.second)
    
    def __sub__(self, num):
        if isinstance(num, int):
            num=dual(num)
        return dual(self.first-num.first, self.second-num.second)
    
    def __mul__(self, num):
        if isinstance(num, int):
            num=dual(num)
        return dual(self.first*num.first, ((self.first*num.second)+(self.second*num.first)))
    
    def __truediv__(self, num):#(self/num)
        if isinstance(num, int):
            num=dual(num)
        mul=self.__mul__(dual(num.first,-num.second))
        num_sq=num.first*num.first
        return dual(mul.first/num_sq, mul.second/num_sq)
    #(a+bε)^(c+dε)
    #(a^c)[1,(dlna+bc/a)]
    def __pow__(self, num): # self to the power num
        if isinstance(num, int):
            num=dual(num)
        if self.first==0:
            return 0
        a_pow_c=self.first**num.first
        return dual(a_pow_c,(a_pow_c)*((num.second*np.log(self.first))+(self.second*num.first)/self.first))
    
    def __log__(self):
        if self.first==0:
            print('natural log not defined for 0.')
            return ValueError
        return dual(np.log(self.first), self.second/self.first)
    
    def __sin__(self):
        return dual(np.sin(self.first), self.second*np.cos(self.first))
    
    def __cos__(self):
        return dual(np.cos(self.first), -self.second*np.sin(self.first))
    
    def __tan__(self):
        return self.__sin__()/self.__cos__()
    
    def show(self):
        l=[self.first, self.second]
        print(l)

In [4]:
#consider the below function, we are going to find its partial derivative with respect to x and y at (3,4)
def f(x,y):
    return x*x*y + y + 2

# 1) partial derivative wrt x at (3,4)
x=dual(3,1)     # x=3+ε
y=dual(4,0)     # y=4
f(x,y).show()

# 2) partial derivative wrt y at (3,4)
x=dual(3,0)     # x=3
y=dual(4,1)     # y=4+ε
f(x,y).show()

[42, 24]
[42, 10]


In [5]:
# partial derivative of f2 wrt x at (2,5)
def f2(x,y):
    return x.__log__()+x*y-y.__tan__()

x=dual(2,1)
y=dual(5,0)
f2(x,y).show()

[14.073662186806532, 5.5]


In [6]:
# finding partial derivative of the sigmoid function
def sig1(inp, weights):
    inp=np.array(inp, dtype=dual)
    weights=np.array(weights, dtype=dual)
    z=np.dot(inp,weights)
    return dual(1)/(dual(2.717).__pow__((z*-1))+1)

inp=[dual(1),dual(2), dual(3)]
weight=[dual(4),dual(5,1), dual(2)]     # d(sig1)/dw2 at w1=4, w2=5, w3=6
np.dot(inp, weight).show()
sig1(inp, weight).show()

[20, 2]
[0.9999999979193106, 4.159415766369578e-09]


In [7]:
a=dual(2,3)
b=dual(4,5)
c=a/b
c.show()

[0.5, 0.125]


In [9]:
c=dual.__add__(a,b)
c.show()

[6, 8]


In [12]:
def func(func1,*args):
    for i in args:
        print(i)
    pass

def func1(*args):
    sum=0
    for i in args:
        sum=sum+i
    return sum

h=func1
g=func(h)
print(g)

type(func.__name__)

None


str